# Car ride-share potential in mid-size U.S. cities from geographic spread (2)

Second notebook for the IBM Data Science Specialization on Coursera, consolidating and cleaning up the first notebook, to continue with less clutter.

## Copy-forward: All imports so far

In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import math
import json

!conda install -c conda-forge geopy=1.18.1 --yes


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge


In [4]:
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.8.0 --yes
import folium


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.8.0                      py_0    conda-forge


In [5]:
import time


## Copy-forward: Mid-size U.S. cities

In [6]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_United_States_cities_by_population&oldid=883568308'
website_url = requests.get(url).text
soup = BeautifulSoup(website_url, 'lxml')
city_table = soup.find('table', { 'class' : 'wikitable sortable' })
print("{}\n\n   [...]\n\n{}".format(str(city_table)[:500].replace('\n', '').replace('<tr>', '\n\n<tr>'), str(city_table)[-500:]))


<table class="wikitable sortable" style="text-align:center"><tbody>

<tr><th>2017<br/>rank</th><th>City</th><th>State<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[5]</a></sup></th><th>2017<br/>estimate</th><th>2010<br/>Census</th><th>Change</th><th colspan="2">2016 land area</th><th colspan="2">2016 population density</th><th>Location</th></tr>

<tr><td>1</td><td style="text-align:left;background-color:#cfecec"><i><a href="/wiki/New_York_City" title="New York 

   [...]

"latitude">38°21′14″N</span> <span class="longitude">121°58′22″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">38.3539°N 121.9728°W</span><span style="display:none">﻿ / <span class="geo">38.3539; -121.9728</span></span><span style="display:none">﻿ (<span class="fn org">Vacaville</span>)</span></span></span></a></span></small>
</td></tr></tbody></table>


In [7]:
l = []

table_rows = city_table.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if len(row) < 1:
        print("(ignoring empty row)")
        test_size = 0
    else:
        test_size = int(row[3].replace(',', ''))
        
    if test_size >= 300000 and test_size <= 400000:
        city_name = re.sub('\[.*\]', '', row[1])
        city_state = row[2]
        city_estd_pop2017 = test_size
        city_latlongraw = re.sub('^.*/', '', re.sub('\(.*\)', '', row[10])).replace(' ', '')
        # strip non-ASCII residue
        city_latlongraw = city_latlongraw.encode('ascii',errors='ignore').decode()
        city_lat = float(re.sub(';.*$', '', city_latlongraw))
        city_long = float(re.sub('^.*;', '', city_latlongraw))
        l.append([city_name, city_state, city_estd_pop2017, city_lat, city_long])

cities_df = pd.DataFrame(l)
cities_df.columns = ['City name', 'City state', 'Population', 'Latitude', 'Longitude']
print(cities_df)


(ignoring empty row)
         City name    City state  Population  Latitude  Longitude
0        Arlington         Texas      396394   32.7007   -97.1247
1      New Orleans     Louisiana      393292   30.0534   -89.9345
2          Wichita        Kansas      390591   37.6907   -97.3459
3        Cleveland          Ohio      385525   41.4785   -81.6794
4            Tampa       Florida      385430   27.9701   -82.4797
5      Bakersfield    California      380874   35.3212  -119.0183
6           Aurora      Colorado      366623   39.6880  -104.6897
7          Anaheim    California      352497   33.8555  -117.7601
8         Honolulu        Hawaii      350395   21.3243  -157.8476
9        Santa Ana    California      334136   33.7363  -117.8830
10       Riverside    California      327728   33.9381  -117.3932
11  Corpus Christi         Texas      325605   27.7543   -97.1734
12       Lexington      Kentucky      321959   38.0407   -84.4583
13        Stockton    California      310496   37.9763 

## Copy-forward: Foursquare functions

In [65]:
# (paste secret from local file)


In [66]:
print('CLIENT_ID set: {}'.format(CLIENT_ID is not None))
print('CLIENT_SECRET set: {}'.format(CLIENT_SECRET is not None))

VERSION = '20180605' # Foursquare API version


CLIENT_ID set: True
CLIENT_SECRET set: True


### >>> getVenuesNearLatLong

In [11]:
def getVenuesNearLatLong(latitude, longitude, radius=500, limit=100, verbose=True):
    
    venues_list=[]
                
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            latitude, 
            longitude, 
            radius, 
            limit)
            
    # make the GET request (on error try four more times before giving up)
    num_tries = 0
    results = [] # assume no venues if persistent error
    
    while num_tries < 5:
        num_tries +=1
        try:
            results_raw = requests.get(url)
            results = results_raw.json()["response"]['groups'][0]['items']
        except:
            print('(err)', end='')
            time.sleep(2) # sleep for two seconds, then retry
        
    # return only relevant information for each nearby venue
    venues_list.append([(
            latitude, 
            longitude, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    if (len(results) > 0):
        nearby_venues.columns = [
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Category']
    
    if verbose:
        print('found {} venues within {} meters of {}/{}'.format(len(results), radius, latitude, longitude))
    else:
        print('{}'.format(len(results)), end='. ')
    
    return(nearby_venues)


### >>> get_venues_in_hex_grid

In [12]:
def get_venues_in_hex_grid(latitude, longitude, venues_dict, this_coord, radius=500, limit=100, new_coords=[], verbose=True):
    '''
    Calls Foursquare in a hex grid around a given coordinate point. If venues
    have already been searched on one of the hex grid points, that result is
    kept and no new search is executed.
    
    Parameters:
    
    latitude and longitude are as of the origin coordinate (0, 0),
    venues_dict are the venues found so far (dictionary keys are a coordinate tuple),
    this_coord is the center coordinate around which the hex grid is to be searched,
    radius is the radius [meters] to search around a coordinate point,
    limit is the maximum number of venues to return from a Foursquare search.
    new_coords is a list of coordinate points that wasn't probed yet
    
    Returns a list of new coordinate tuples appended to the new_coords parameter, if any
    '''
    
    r_earth = 6378000. # approximate radius of the Earth in meters
    pi = math.pi
    sqrt_three = math.sqrt(3.)
    overlap = 1.4 # 40% overlap
    
    cx = this_coord[0] # center X
    cy = this_coord[1] # center Y
    hex_coords = [ (cx-1,cy), (cx+1, cy), (cx,cy-1), (cx,cy+1), (cx-1,cy+1), (cx+1,cy-1) ] # the gex grid around this_coord
    
    if (cx, cy) in new_coords:
        new_coords.remove((cx, cy))
    
    for this_hex in hex_coords:
        if not this_hex in venues_dict:
            # the coordinate has not been searched for
            
            # get the x- and y-step from a hex grid; start with a square grid (letting the circles overlap a bit):
            dx_square = this_hex[0] * radius * ( overlap / 2. )
            dy_square = this_hex[1] * radius * ( overlap / 2. )
            # now convert to a hex grid:
            dx = dx_square + dy_square / 2.
            dy = dy_square * ( sqrt_three / 2. )
            # approximate the center point's latitude and longitude assuming locally flat Earth
            hex_latitude  = latitude  + (dy / r_earth) * (180 / pi);
            hex_longitude = longitude + (dx / r_earth) * (180 / pi) / math.cos(latitude * pi/180);
            
            if verbose:
                print('getting coordinate {}...'.format(this_hex))
            else:
                print('({},{}):'.format(this_hex[0], this_hex[1]), end='')
                
            this_venues = getVenuesNearLatLong(hex_latitude, hex_longitude, radius=radius, limit=limit, verbose=verbose)
            venues_dict[this_hex] = this_venues
            if not this_hex in new_coords:
                new_coords.append(this_hex)
    
    return new_coords


### >>> make_map_from_dict

In [14]:
def make_map_from_dict(lat_orig, long_orig, venues_dict, zoom_start,
                       city_name='(city_name)', city_state='(city_state)',
                       radius_zoom=1.0, width=600, height=600,
                       no_touch=True,
                       zoom_control=False):
    
    if zoom_control == False:
        min_zoom = zoom_start
        max_zoom = zoom_start
    else:
        min_zoom = 0
        max_zoom = 16
    
    new_map = folium.Map(
        location=[lat_orig, long_orig],
        zoom_start=zoom_start,
        width=width,
        height=height,
        control_scale=True,
        no_touch=no_touch,
        min_zoom=min_zoom,
        max_zoom=max_zoom
    )

    # add markers to map
    for coords, venues in venues_dict.items():
        if venues.shape[0] > 0: 
            label = '{}, # venues: {}'.format(coords, venues.shape[0])
            label = folium.Popup(label, parse_html=True)
            folium.CircleMarker(
                [venues['Latitude'][0], venues['Longitude'][0]],
                radius=venues.shape[0] * radius_zoom,
                popup=label,
                color='blue',
                fill=True,
                fill_color='#3186cc',
                fill_opacity=0.7,
                parse_html=False).add_to(new_map)
        
    legend_html = ('<div style="position: fixed; top: 30px; left: 50px; width: 450px;' 
                + 'height: 30px; border: 2px solid grey; z-index: 9999; font-size: 16px; background-color: white">' 
                + '&nbsp;{},&nbsp;{}' 
                + '</div>').format(
                     city_name,
                     city_state
                     )
    new_map.get_root().html.add_child(folium.Element(legend_html))
     
    return new_map

### >>> find_venues_geo_distribution

In [15]:
def find_venues_geo_distribution(cities_df, city_index, max_coords_tested=100, radius=1500, limit=100, verbose=True):
    
    city_name = cities_df['City name'][city_index]
    city_state = cities_df['City state'][city_index]
    
    # initialize venues_dict with the venues dataframe at the origin
    venues_dict = {}
    origin_coord = (0,0)
    lat_orig = cities_df.Latitude[city_index]
    long_orig = cities_df.Longitude[city_index]
    print('[test #1 of {}] (0,0):'.format(max_coords_tested), end='')
    venues_df = getVenuesNearLatLong(lat_orig, long_orig, radius=radius, verbose=verbose)
    venues_dict[origin_coord] = venues_df
    
    # mark the origin as the first (and only) coordinate point not yet explored
    new_coords = [(0, 0)]
    num_coords_tested = 1
    
    while num_coords_tested < max_coords_tested:
        
        highest_venues = -1
        new_test_coord = None
        
        for this_coord in new_coords:
            venues_df = venues_dict[this_coord]
            if venues_df.shape[0] > highest_venues:
                new_test_coord = this_coord
                highest_venues = venues_df.shape[0]
        
        # call the hex grid exploration function
        num_coords_tested += 1
        print('[test #{} of {}]'.format(num_coords_tested, max_coords_tested), end=' ')
        new_coords = get_venues_in_hex_grid(lat_orig, long_orig, venues_dict, new_test_coord, radius=radius, new_coords=new_coords, limit=limit, verbose=verbose )
        
    return lat_orig, long_orig, venues_dict, city_name, city_state


## Determine aggregates describing the geographical distribution

In order to cluster cities by shape of their venues, we need to calculate aggregate variables:

* Determine the center of the venues by averaging each coordinate point, weighed by the number of venues there.
* Determine the mean distance of all venues from that center.
* Determine the standard deviation of the distance distribution, as well as skewedness and kurtosis ("peaky-ness"). Once we have these values, all details on the geographic venues distribution will be discarded.

But first we have to simplify the dataframe, to sum up the number of indicator venues as compared to the total number of venues.

### Check whether a venue is of an indicator type

Define a vendor indicator function now so that the remainder of the algorithm can be developed and tested. For the sake of this capstone assignment, I'll simply assume to make a study for a client, and the client gave me that list. It is aimed at indicating a population that is willing to try out new and unusual things, in the context of sharing transportation.

### >>> is_indicator_venue

In [38]:
def is_indicator_venue(venue_type):
    if venue_type is None:
        return False
    
    magic_words = [
        'yoga',
        'salad',
        'coworking',
        'alternative',
        'bike',
        'fitness',
        'running',
        'jogging',
        'cycling',
        'cycle',
        'athletics',
        'gluten',
        'health',
        'recreation',
        'tennis',
        'vegetarian',
        'vegan',
        'tennis',
        'disc golf',
        'pilates',
        'share',
        'sharing',
        'incubat',
        'innovat'
    ]
    return any(substring in venue_type.lower() for substring in magic_words)

print('Perform some tests on indicator venues:')
print(is_indicator_venue(None))
print(is_indicator_venue(''))
print(is_indicator_venue('Salad Bar'))
print(is_indicator_venue('Chinese Restaurant'))
print(is_indicator_venue('Coworking space'))


Perform some tests on indicator venues:
False
False
True
False
True


### Aggregate venues_dict into a DataFrame with one row per coordinate

For all coordinates, count the number of venues at that coordinate as well as the number of indicator venues

### >>> aggregate_venues_dict

In [89]:
def aggregate_venues_dict(venues_dict):
    venues_agg = []
    venues_types = []
    for coord, venues in venues_dict.items():
        num_venues = venues.shape[0]
        num_indicators = 0
        if num_venues > 0:
            for this_type in venues['Venue Category'].values:
                venues_types.append(this_type)
                if is_indicator_venue(this_type):
                    num_indicators += 1
        venues_agg.append([coord, num_venues, num_indicators])
    venues_agg_df = pd.DataFrame(venues_agg)
    venues_agg_df.columns = ['coord', 'num_venues', 'num_indicators']
    return venues_agg_df, set(venues_types)


Do a quick test: Get 10 coordinates (plus a few surrounding ones) from the third city in the list (index 2):

In [90]:
lat_orig, long_orig, venues_dict, city_name, city_state = find_venues_geo_distribution(
    cities_df, city_index=2, radius=800, max_coords_tested=10, verbose=False)


[test #1 of 10] (0,0):43. [test #2 of 10] (-1,0):33. (1,0):46. (0,-1):58. (0,1):18. (-1,1):16. (1,-1):58. [test #3 of 10] (-1,-1):44. (0,-2):43. (1,-2):53. [test #4 of 10] (2,-1):78. (2,-2):55. [test #5 of 10] (3,-1):87. (2,0):83. (3,-2):73. [test #6 of 10] (4,-1):74. (3,0):79. (4,-2):52. [test #7 of 10] (2,1):36. (1,1):28. [test #8 of 10] (4,0):55. (3,1):29. [test #9 of 10] (5,-1):24. (5,-2):24. [test #10 of 10] (3,-3):29. (4,-3):16. 

In [91]:
print("{}, {}".format(city_name, city_state))
venues_agg_df, venues_types = aggregate_venues_dict(venues_dict)
venues_agg_df.head(10)


Wichita, Kansas


,coord,num_venues,num_indicators
0,"(5, -1)",24,1
1,"(5, -2)",24,1
2,"(0, 0)",43,4
3,"(-1, 0)",33,1
4,"(3, -3)",29,0
5,"(3, 0)",79,2
6,"(2, -1)",78,1
7,"(2, 1)",36,1
8,"(3, -2)",73,0
9,"(2, -2)",55,0


The `aggregate_venues_dict` function also returns a set of all venues, let's loop through all of them and test the indicator function.

In [92]:
for this_type in venues_types:
    print('{}:  {}'.format(is_indicator_venue(this_type), this_type))
    

False:  Social Club
False:  Bar
False:  Discount Store
False:  Design Studio
False:  Hot Dog Joint
False:  Grocery Store
False:  Smoothie Shop
False:  Science Museum
True:  Vegetarian / Vegan Restaurant
False:  Hobby Shop
False:  Lawyer
False:  Museum
False:  Vietnamese Restaurant
False:  Sushi Restaurant
False:  BBQ Joint
False:  American Restaurant
False:  Asian Restaurant
False:  Neighborhood
False:  Hotel
False:  Sandwich Place
False:  Chinese Restaurant
False:  Monument / Landmark
False:  Baseball Stadium
False:  Breakfast Spot
False:  History Museum
False:  Taco Place
False:  Steakhouse
False:  Rental Car Location
False:  Cosmetics Shop
False:  Sporting Goods Shop
False:  Skate Park
False:  Diner
False:  Record Shop
False:  Bistro
False:  Nightclub
False:  Pizza Place
False:  French Restaurant
False:  Multiplex
False:  Liquor Store
False:  Sports Bar
False:  Plaza
False:  Cocktail Bar
False:  Candy Store
False:  Fast Food Restaurant
False:  Video Store
False:  Frozen Yogurt Shop


Works.

### Find coordinate center of all venues

In order to analyze the geographic spread of the venues, the center of the venues has to be determined first. This is done by averaging all coordinates, weighted by the number of venues there.

### >>> find_venues_distance_to_center

In [93]:
def find_venues_distance_to_center(venues_agg_df):
    '''
    Appends or updates x/y and distance-to-center columns of venues_agg_df.
    Also returns some aggregates that were calculated along the way.
    '''
    
    x, y = zip(*venues_agg_df['coord'])
    total_num_venues = venues_agg_df['num_venues'].sum()
    total_num_indicators = venues_agg_df['num_indicators'].sum()
    center_x = ( x * venues_agg_df['num_venues'] ).sum() / total_num_venues
    center_y = ( y * venues_agg_df['num_venues'] ).sum() / total_num_venues
    dx = x - center_x
    dy = y - center_y
    dist2 = dx * dx + dy * dy
    venues_agg_df['dist'] = np.sqrt(dist2)
    
    return center_x, center_y, total_num_venues, total_num_indicators


In [94]:
# test the function
center_x, center_y, total_num_venues, total_num_indicators = find_venues_distance_to_center(venues_agg_df)
print('Test set coordinate center of {0:}, {1:} is at ({2:.2f}, {3:.2f}). Indicator / total venues: {4:} / {5:} ({6:.1f}%)'.format(
    city_name, city_state, center_x, center_y, total_num_indicators, total_num_venues,
    total_num_indicators * 100 / total_num_venues))
print('\nFirst five Euclidean coordinate distances to the center:\n\n{}'.format(venues_agg_df.head(5)))



Test set coordinate center of Wichita, Kansas is at (2.04, -0.84). Indicator / total venues: 24 / 1234 (1.9%)

First five Euclidean coordinate distances to the center:

     coord  num_venues  num_indicators      dist
0  (5, -1)          24               1  2.966477
1  (5, -2)          24               1  3.182609
2   (0, 0)          43               4  2.202692
3  (-1, 0)          33               1  3.150877
4  (3, -3)          29               0  2.368620


### Gather distribution statistics

Now gather ... TODO TBD something

### >>> gather_distribution_stats

In [95]:
def gather_distribution_stats(venues_agg_df):
    center_x, center_y, total_num_venues, total_num_indicators = find_venues_distance_to_center(venues_agg_df)
    
    